In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import svm
from scipy.stats import poisson, binom
from cpp.optimize_price import PriceOptimizer as PriceOptimizer_CPP
from python.optimize_price import PriceOptimizer as PriceOptimizer_Python
import time

In [ ]:
# Generate training data for sales probability regression
def rank(a, p):
    _rank = p.shape[0]
    for i in range(p.shape[0]):
        if a < p[i]:
            _rank = _rank - 1
    return _rank

def make_X(price, competitor_prices, t):
    return np.array([
            rank(price, competitor_prices),
            price - competitor_prices.min(),
            competitor_prices.size,
            (price + np.sum(competitor_prices)) / (1 + competitor_prices.size),
            t,
            t * t
        ])

def generate_train_data(B=1000, T=30):
    our_price = np.around(10 + np.random.uniform(0, 10, (B * T, 1)), decimals=2)
    competitor_prices = np.around(10 + np.random.uniform(0, 10, (B * T, 5)), decimals=2)
    
    X = np.zeros((B * T, 6))
    Y = np.zeros(B * T)
    for t in range(T):
        for i in range(B):
            index = t * B + i
            X[index,:] = make_X(our_price[index], competitor_prices[index], t)
            _rank = rank(our_price[index], competitor_prices[index])
            # Y[index] = _rank == 0
            # Y[index] = np.round(np.random.uniform(0, 1)
            #                                   * (0.5 + 0.5 * (1 - _rank / 5))
            #                                   * (0.5 + 0.5 * t / T))
            Y[index] = np.round(np.random.uniform(0.5 * t / T, 1) * (1 - _rank / 5))

    return (X, Y)

X, Y = generate_train_data()

In [ ]:
# Train sales probability model
def make_model(X, Y):
    regr = linear_model.LogisticRegression()
    regr.fit(X, Y)
    def predict(x):
        # return np.maximum(0, regr.predict(x))
        return regr.predict_proba(x)[:, 1]
    coef = np.concatenate((regr.intercept_.ravel(), regr.coef_.ravel()))
    return predict, coef

sales_model, sales_model_coef = make_model(X, Y)

In [ ]:
# Plot sales probability model
X_test, _ = generate_train_data(1000)
X_test = np.sort(X_test.view('f8,f8,f8,f8,f8,f8'), order=['f0', 'f4'], axis=0).view(float)
plt.scatter(np.arange(0, X_test.shape[0]), X_test[:, 0], color='black')
plt.plot(np.arange(0, X_test.shape[0]), sales_model(X_test), color='blue', linewidth=1)

plt.show()

In [ ]:
def timeit(func):
    start = time.clock()
    func()
    end = time.clock()
    print(end - start, 's')
    
def plot(po, t, n):
    po.run(t, n)
    for i_n in range(1, n + 1):
        n_data = []
        for i_t in range(1, t + 1):
            n_data.append(po.run(i_t, i_n)[0])
        if i_n in [1, 2, 3, 4, 5, 10, 20, 30]:
            label_text = "N=" + str(i_n)
            plt.plot(n_data, label=label_text)
    plt.ylabel('n')
    plt.xlabel('t')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
        
def make_price_optimizer(sales_model_coef, competitor_prices,
               T=20, N=15, M=5,
               price_range=np.arange(10, 20, 1), 
               L=0.01, delta=0.99, Z=0.5):
    po = PriceOptimizer_CPP(T, N)
    # po = PriceOptimizer_Python(sales_model_coef, competitor_prices, T, N)
    # po.make_X = make_X
    po.L = L
    po.Z = Z
    po.M = M
    po.delta = delta
    po.price_range = price_range
    po.competitor_prices = competitor_prices
    po.sales_model_coef = sales_model_coef
    return po

In [ ]:
def simulation(sales_model, competitor_prices, T=20, N=15, iterations=20):
    
    def sales_prob(price, t):
        x = make_X(price, competitor_prices, t).reshape(1, -1)
        return sales_model(x)[0]
    
    profits = []
    
    L=0.01
    Z=0.5
    M=2

    optimizer = make_price_optimizer(sales_model_coef, competitor_prices, 
                               T=T, N=N, M=M, L=L, Z=Z, price_range=np.arange(10., 20., 0.2))
    
    timeit(lambda: plot(optimizer, T, N))
    
    for i in range(iterations):
        
        price_history = []
        accumulated_sales = []
    
        profit = 0
        n = N
        
        for t in range(0, T):
            price, V = optimizer.run(t, n)
            pi = sales_prob(price, t)
            sales = min(n, np.random.poisson(M * pi))
            # sales = min(n, np.random.binomial(M, pi))
            n = n - sales
            profit += price * sales - L * n

            price_history.append(price)
            accumulated_sales.append(N - n)

            # # Change competitor prices
            # competitor_prices = competitor_prices * np.random.uniform(0.8, 1.2, 5)
            # optimizer = PriceOptimizer(sales_model, competitor_prices, N=n, T=T, L=L, Z=Z)

        # Realize salvage profits
        profit += n * Z

        if i < 5:
            print("Estimated profit: {0}".format(optimizer.run(0, N)[1]))
            print("Simulated profit: {0}".format(profit))
            
            plt.plot(price_history)    
            plt.ylabel('Price Path')
            plt.show()

            plt.plot(accumulated_sales)    
            plt.ylabel('Accumulated Sales')
            plt.show()
        
        profits.append(profit)

    return profits

# competitor_prices_simulation = 16 + np.random.uniform(0, 4, 5)
# simulation(sales_model, competitor_prices_simulation)

In [ ]:
# competitor_prices = np.around(10 + np.random.uniform(0, 10, 5), decimals=2)
competitor_prices = np.array([11., 13., 14., 16., 18.])
print(competitor_prices)
profits = simulation(sales_model, competitor_prices, T=100, N=100, iterations=1000)
    
n, bins, patches = plt.hist(profits, 10, normed=1, facecolor='green', alpha=0.75)

plt.xlabel('Profit')
plt.ylabel('Probability')
plt.show()